# Группировки как фильтр
Считаем метрики для пользователей, которые выставили 100 и более оценок

In [ ]:
import pandas as pd

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

### Упражнение
Получите список пользователей film_fans_user_ids, которые выставили 100 и более оценок фильмам:
1. Для каждого userId посчитайте количество выставленных оценок. Результат сохраните в датафрейм ratings_grouped
2. Отфильтруйте пользователей в ratings_grouped, которые поставили 100 и более оценок. Результат сохраните в датафрейм ratings_filtered
3. Сохраните идентификаторы пользователей датафрейма ratings_filtered в список film_fans_user_ids:
```python
film_fans_user_ids = ratings_filtered['userId'].tolist()
```
4. Отфильтруйте исходный датафрейм ratings для пользователей из списка film_fans_user_ids:
```python
ratings[ ratings['userId'].isin(...) ]
```

# Сводные таблицы
Прям как в экселе

In [ ]:
data = pd.read_csv('ml-latest-small/ratings.csv')
data.head()

In [ ]:
# какие оценки поставил пользователь №1

data[ data['userId'] == 1 ]['rating'].value_counts()

In [ ]:
# а для всех пользователей сразу?

data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)

In [ ]:
# можно итоги добавить
data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0,
                 margins = True)

### Упражнение
Какой пользователь выставил больше всех пятерок?

# Объединение датафреймов

In [ ]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

In [ ]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

In [ ]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

In [ ]:
visits.groupby('user_id').count().reset_index()

In [ ]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

In [ ]:
purchases_pivot.reset_index()

In [ ]:
visits_grouped.join(purchases_pivot)

### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [ ]:
visits_grouped.join(purchases_pivot, 
                    how='left')

### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [ ]:
visits_grouped.join(purchases_pivot, 
                    how='right')

### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают расходы.

In [ ]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [ ]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

In [ ]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

In [ ]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [ ]:
visits_grouped.join(purchases_pivot, 
                    how='inner')

### Outer join
Оставляет все строчки

In [ ]:
visits_grouped.join(purchases_pivot, 
                    how='outer')

### Дубликаты при объединении таблиц

In [ ]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

In [ ]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

In [ ]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

In [ ]:
movies.drop_duplicates(subset = ['movieId'], keep = 'first', inplace = True)
movies.head()

In [ ]:
ratings.merge(movies, how = 'left', on = 'movieId')

In [ ]:
ratings.merge(movies, how = 'right', on = 'movieId')

### Оптимизация хранения данных

In [ ]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [ ]:
len(ratings) == len(joined)

In [ ]:
joined.to_csv('joined_ratings.csv', index=False)

### Какой жанр имеет самые высокие рейтинги?

In [ ]:
import pandas as pd

In [ ]:
genres = ['Drama', 'Action', 'Thriller']

In [ ]:
import numpy as np

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

In [ ]:
len(movies)

In [ ]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

In [ ]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

### Считаем рейтинг жанров

In [ ]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base